# Script para ler o Diário Oficial do Município de São Paulo

## Script BASE

### Importando as bibliotecas

In [252]:
from datetime import date, datetime, timedelta
import os
import fitz as fz
import tabula

### Obter data atual

In [253]:
def get_date_today():
    '''Obtém a data atual e retorna uma string: (YYYY-mm-dd)'''
    dt_now = date.today()
    if dt_now.weekday() == 0:
            dt_now = dt_now + timedelta(-2)
            dt_now = dt_now.strftime("%Y-%m-%d")
    else:
            dt_now = dt_now.strftime("%Y-%m-%d")
    
    return dt_now

dt_now = get_date_today()

### Obtendo diretórios

In [254]:
def get_directories():
        '''Obtém  os diretórios de trabalho.
                return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES
        '''
        dt_now = get_date_today()
        BASE_DIR = os.getcwd()
        os.chdir(BASE_DIR)
        try:
                os.makedirs(f'{BASE_DIR}\\outputs_files\\{dt_now}')
        except:
                pass

        DATA_DIR = f'{BASE_DIR}\\querido-diario\\data_collection\\data\\3550308\\{str(dt_now)}'
        SAVE_DIR = f'{BASE_DIR}\\outputs_files\\{dt_now}'
        DATA_FILE = SAVE_DIR + '\\' + [f for f in os.listdir(SAVE_DIR) if f.endswith("tsv")][0]
        DATA_FILES = os.listdir(DATA_DIR)
        
        print(f'### INFO:  DIRS ###')
        print(f'BASE_DIR:  {BASE_DIR}')
        print(f'DATA_DIR:  {DATA_DIR}')
        print(f'SAVE_DIR:  {SAVE_DIR}')
        print(f'DATA_FILE  {DATA_FILE}')
        
        return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES

BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES = get_directories()

### INFO:  DIRS ###
BASE_DIR:  c:\Projetos\scrapy_dom
DATA_DIR:  c:\Projetos\scrapy_dom\querido-diario\data_collection\data\3550308\2022-05-25
SAVE_DIR:  c:\Projetos\scrapy_dom\outputs_files\2022-05-25
DATA_FILE  c:\Projetos\scrapy_dom\outputs_files\2022-05-25\DOM_SP-67_97_104_2022-05-25.tsv


### Obtendo Informações do DOM

In [255]:
def get_info_dom():
        '''Obtém as informações do DOM.
                return YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM        
        '''
        dt_now = get_date_today()
        BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES = get_directories()
        YEAR_DOM = date.today().year - 1955
        HEAD_DOM ='São Paulo, ' + str(YEAR_DOM) + ' ('

        with fz.open(DATA_DIR +'\\'+ DATA_FILES[3]) as file:
                search_edition = ''
                for page in file:
                        search_edition += page.get_text()

        search_edition = search_edition[search_edition.find(HEAD_DOM):]
        EDITION_DOM = search_edition[search_edition.find('(')+1:search_edition.find(')')]
        HEAD_DOM ='São Paulo, ' + str(YEAR_DOM) + ' (' + EDITION_DOM + ')'
        PAGS_DOM = str(len(DATA_FILES))

        print('### INFO: DOM-SP ###')
        print(f'DATA: {dt_now}')
        print(f'PAGS: {PAGS_DOM}')
        print(f'HEAD: {HEAD_DOM}')

        return YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM

YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM = get_info_dom()

### INFO:  DIRS ###
BASE_DIR:  c:\Projetos\scrapy_dom
DATA_DIR:  c:\Projetos\scrapy_dom\querido-diario\data_collection\data\3550308\2022-05-25
SAVE_DIR:  c:\Projetos\scrapy_dom\outputs_files\2022-05-25
DATA_FILE  c:\Projetos\scrapy_dom\outputs_files\2022-05-25\DOM_SP-67_97_104_2022-05-25.tsv
### INFO: DOM-SP ###
DATA: 2022-05-25
PAGS: 104
HEAD: São Paulo, 67 (97)


### Salva arquivo final

In [256]:
def save_file(SAVE_DIR,CONTEXT,NAME_FILE,FORMAT='txt'):
        '''
                Cria um arquivo com o conteúdo e salva na pasta.
                        SAVE_DIR  : Diretório para salvar o conteúdo
                        CONTEXT   : Conteúdo a ser salvo
                        NAME_FILE : Nome do arquivo para ser salvo
                        FORMAT    : Formato do arquivo final
        '''

        with open(f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}","w",encoding='utf8') as output_file:
                output_file.write(CONTEXT)
        
        return f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}"

### Obtem contéudo do arquivo TSV

In [257]:
with open(DATA_FILE,'r') as file:
    conteudo = file.read()

## Scripts de Listagens

### Seção SERVIDORES

In [258]:
# Localiza o início das seções SERVIDORES e CONCURSOS
sc_servidores = conteudo.find('\nSERVIDORES\n')
sc_concursos = conteudo.find('\nCONCURSOS\n')
print(f"SERVIRODRES: {sc_servidores} / CONCURSOS: {sc_concursos}")
conteudo_servidores = conteudo[sc_servidores:sc_concursos]

SERVIRODRES: 677493 / CONCURSOS: 1031372


#### Limpeza dos dados

In [279]:
conteudo_servidores = conteudo_servidores.replace('\t',' ')
conteudo_servidores = conteudo_servidores.replace('""','')
conteudo_servidores = conteudo_servidores.replace(' \n','\n')
conteudo_servidores = conteudo_servidores.replace('\n ','\n')
conteudo_servidores = conteudo_servidores.replace('    ',' ')
conteudo_servidores = conteudo_servidores.replace('   ',' ')
conteudo_servidores = conteudo_servidores.replace('  ',' ')
conteudo_servidores = conteudo_servidores.replace('-\n','')
conteudo_servidores = conteudo_servidores.replace(',\n',', ')
conteudo_servidores = conteudo_servidores.replace(', \n',', ')
conteudo_servidores = conteudo_servidores.upper()
conteudo_servidores = conteudo_servidores.replace('FUNDAÇÃO PAULISTANA DE EDUCAÇÃO\nTECNOLOGIA E CULTURA','FUNDAÇÃO PAULISTANA DE EDUCAÇÃO TECNOLOGIA E CULTURA')
conteudo_servidores = conteudo_servidores.replace('HOSPITAL DO SERVIDOR\nPÚBLICO MUNICIPAL','HOSPITAL DO SERVIDOR PÚBLICO MUNICIPAL')
conteudo_servidores = conteudo_servidores.replace('SECRETARIA MUNICIPAL DE ASSIST E DESENV\nSOCIAL','SECRETARIA MUNICIPAL DE ASSIST E DESENV SOCIAL')
conteudo_servidores = conteudo_servidores.replace('SECRETARIA MUNICIPAL DE INFRAESTRUTURA\nURBANA E OB','SECRETARIA MUNICIPAL DE INFRAESTRUTURA URBANA E OB')
conteudo_servidores = conteudo_servidores.replace('SECRETARIA MUNICIPAL DE SEGURANCA\nURBANA','SECRETARIA MUNICIPAL DE SEGURANCA URBANA')
conteudo_servidores = conteudo_servidores.replace('SECRETARIA MUNICIPAL DO VERDE E DO MEIO\nAMBIENTE','SECRETARIA MUNICIPAL DO VERDE E DO MEIO AMBIENTE')
conteudo_servidores = conteudo_servidores.replace('LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS\nEM GRAU DE RECURSO','LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS\nNOS TERMOS DA LEI 8989/79','RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA AVALIAÇÃO DE ESPECIALISTA','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA JUNTA MÉDICA PARA FINS DE PENSÃO\nPOR MORTE','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM\nFUNÇÃO DE:','RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM FUNÇÃO DE:')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE SERVIDORES SUBMETIDOS À\nAVALIAÇÃO DE ESPECIALISTA\nCOM CAPACIDADE LABORATIVA PARA A SUA\nFUNÇÃO ORIGINAL','RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE ESPECIALISTA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO ORIGINAL')
conteudo_servidores = conteudo_servidores.replace('LICENÇAS MÉDICAS DE CURTA DURAÇÃO\nDOC ','LICENÇAS MÉDICAS DE CURTA DURAÇÃO DOC ')
conteudo_servidores = conteudo_servidores.replace('A PARTIR DE',' A_PARTIR_DE')
conteudo_servidores = conteudo_servidores.replace('ACI-\nDENTE','ACIDENTE')
conteudo_servidores = conteudo_servidores.replace('CANCELA-\nDAS','CANCELADAS')
conteudo_servidores = conteudo_servidores.replace('RETIFICA-\nDAS','RETIFICADAS')
conteudo_servidores = conteudo_servidores.replace('RECO-\nMENDAÇÃO','RECOMENDAÇÃO')
conteudo_servidores = conteudo_servidores.replace('RECOMENDAÇÃO\nD','RECOMENDAÇÃO D')
conteudo_servidores = conteudo_servidores.replace('LICENÇAS MÉDICAS\nFACE','LICENÇAS MÉDICAS FACE')
conteudo_servidores = conteudo_servidores.replace('LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA\nPREVIDÊNCIA SOCIAL\nCONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO\n','LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA PREVIDÊNCIA SOCIAL CONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO ')
conteudo_servidores = conteudo_servidores.replace('DECRETO NO\n','DECRETO NO')
conteudo_servidores = conteudo_servidores.replace('PUBLICADO\nNO','PUBLICADO NO')
conteudo_servidores = conteudo_servidores.replace('\nCONCEDIDA',' CONCEDIDA')
conteudo_servidores = conteudo_servidores.replace('\nCONCEDIDAS ',' CONCEDIDAS ')
conteudo_servidores = conteudo_servidores.replace(' DA LEI\n',' DA LEI ')
conteudo_servidores = conteudo_servidores.replace('\nAOS ',' AOS ')
conteudo_servidores = conteudo_servidores.replace(' SÃO\n',' SÃO ')
conteudo_servidores = conteudo_servidores.replace(' À\n',' À ')
conteudo_servidores = conteudo_servidores.replace(' ÀS\n',' ÀS ')
conteudo_servidores = conteudo_servidores.replace(' AS\n',' AS ')
conteudo_servidores = conteudo_servidores.replace(' E\n',' E ')
conteudo_servidores = conteudo_servidores.replace(' O\n',' O ')
conteudo_servidores = conteudo_servidores.replace(' DA\n',' DA ')
conteudo_servidores = conteudo_servidores.replace(' DAS\n',' DAS ')
conteudo_servidores = conteudo_servidores.replace(' DA(S)\n',' DA(S) ')
conteudo_servidores = conteudo_servidores.replace(' DE\n',' DE ')
conteudo_servidores = conteudo_servidores.replace(' DO\n',' DO ')
conteudo_servidores = conteudo_servidores.replace(' DOS\n',' DOS ')
conteudo_servidores = conteudo_servidores.replace(' DO(S)\n',' DO(S) ')
conteudo_servidores = conteudo_servidores.replace(' COM\n',' COM ')
conteudo_servidores = conteudo_servidores.replace(' EM\n',' EM ')
conteudo_servidores = conteudo_servidores.replace(' PARA\n',' PARA ')
conteudo_servidores = conteudo_servidores.replace(' NA\n',' NA ')
conteudo_servidores = conteudo_servidores.replace(' NAS\n',' NAS ')
conteudo_servidores = conteudo_servidores.replace(' NO\n',' NO ')
conteudo_servidores = conteudo_servidores.replace(' NOS\n',' NOS ')
conteudo_servidores = conteudo_servidores.replace('RF NOME DIAS A PARTIR DE ARTIGO','RF NOME DIAS A_PARTIR_DE ARTIGO')
conteudo_servidores = conteudo_servidores.replace('REG. FUNC. NOME CONTROLE NO ','RF NOME CONTROLE_NO\n')
conteudo_servidores = conteudo_servidores.replace('R.F./V NOME DIAS/EXERCÍCIO A_PARTIR_DE','RF/V NOME DIAS/EXERCÍCIO A_PARTIR_DE')
conteudo_servidores = conteudo_servidores.replace('REG. FUNC/VINC NOME NÍVEL CAT. SÍMBOLO A_PARTIR_DE','RF/v NOME NÍVEL_CAT SÍMBOLO A_PARTIR_DE')
conteudo_servidores = conteudo_servidores.replace('RF V NOME EH A PARTIR MOTIVO','RF VINC NOME EH A_PARTIR MOTIVO')

In [280]:
#print(conteudo_servidores)
save_file(SAVE_DIR,conteudo_servidores,'seção_servidores')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-25\\seção_servidores.txt'

In [281]:
#print(conteudo_servidores)

### Relação de Licença Médica

#### CONTEÚDO PRINCIPAL

In [311]:
conteudo_licenca_medica = conteudo_servidores

sc_lm = conteudo_licenca_medica.find('\nRELAÇÃO DE LICENÇA MÉDICA')
conteudo_licenca_medica = conteudo_licenca_medica[sc_lm:]
sc_lm

21588

In [312]:
i = 0
rl_lm =  conteudo_licenca_medica.split('\n')
licencas = {}
listas = []

for line in rl_lm:
    if 'LICENÇAS MÉDICAS' in line and not 'SEÇÃO DE LICENÇAS MÉDICAS' in line:
        licencas.update({
            line : i
        })
        listas.append(i)
    elif 'RELAÇÃO DE' in line and not 'RELAÇÃO DE LICENÇA MÉDICA' in line:
        licencas.update({
            line : i
        })
        listas.append(i)
    
    i += 1

print(licencas)
print(listas)
len(f'Total de listas: {len(listas)}')
save_file(SAVE_DIR,str(licencas),'listagens_dom')

{'RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79': 4, 'RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79': 724, 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE': 761, 'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA': 764, 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA': 777, 'RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA': 781, 'RELAÇÃO DE LICENÇAS MÉDICAS CANCELADAS EM FUNÇÃO DE:': 799, 'RELAÇÃO DE LICENÇAS MÉDICAS RETIFICADAS EM FUNÇÃO DE:': 805, 'ENGLOBAMENTO DE LICENÇAS MÉDICAS FACE PORTARIA 11/84': 824, 'RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM FUNÇÃO DE:': 873, 'RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO': 878, 'RELAÇÃO DE SERVIDORES COM ALTERAÇÕES NAS DATAS DE ATENDIMENTO PARA AVALIAÇÃO MÉDICA DE ACIDENTE DE TRABALHO': 892, 'FÉRIAS, LICENÇAS MÉDICAS, LICENÇA GALA, ALTERAÇÃO DE ESTADO CIVIL, ALTERAÇÃO DE NOME E LICENÇA NOJO.': 1385, 'LICENÇAS MÉDICAS DE CURTA DURAÇÃO': 1394,

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-25\\listagens_dom.txt'

#### RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79

In [313]:
RELACAO = 'RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [249]:
i = 0
SECRETARIA = ''
str_list_items = 'RF;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric():
        #print(s_item[0])
        #print(s_item[1:-3])
        #print(s_item[-3])
        #print(s_item[-2])
        #print(SECRETARIA)
        #print(s_item[-1])
        if s_item[-3].isnumeric():
            nome = str(' ').join(s_item[1:-3])
            str_list_items += f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}\n'
            #print(str_list_items)
        else:
            nome = str(' ').join(s_item[1:-2])
            str_list_items += f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}\n'
            #print(str_list_items)

    i += 1

In [272]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-25\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_NOS_TERMOS_DA_LEI_8989_79.csv'

#### RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79

In [273]:
RELACAO = 'RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#lm_8989_negada


In [274]:
i = 0
SECRETARIA = ''
str_list_items = 'RF;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric():
        #print(s_item[0])
        #print(s_item[1:-3])
        #print(s_item[-3])
        #print(s_item[-2])
        #print(SECRETARIA)
        #print(s_item[-1])
        nome = str(' ').join(s_item[1:-3])
        #print(f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}\\n')
        str_list_items += f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}\n'
    i += 1

In [275]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-25\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_NEGADAS_NOS_TERMOS_DA_LEI_8989_79.csv'

#### RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE

In [290]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#lm_faltas_pensao

In [301]:
i = 0
SECRETARIA = ''
str_list_items = 'RF;NOME;A_PARTIR_DE;SECRETARIA;LISTA\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 3 and s_item[0][0].isnumeric():
        #print(s_item[0])
        #print(s_item[1:-3])
        #print(s_item[-3])
        #print(s_item[-2])
        #print(SECRETARIA)
        #print(s_item[-1])
        if s_item[-1][0].isnumeric():
            nome = str(' ').join(s_item[1:-1])
            str_list_items += f'{s_item[0]};{nome};{s_item[-1]};{SECRETARIA};{RELACAO}\n'
            #print(str_list_items)
        else:
            nome = str(' ').join(s_item[1:-2])
            str_list_items += f'{s_item[0]};{nome};{s_item[-1]};{SECRETARIA};{RELACAO}\n'
            #print(str_list_items)
    i += 1

In [302]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-25\\RELAÇÃO_DE_FALTAS_DOS_CONVOCADOS_PARA_JUNTA_MÉDICA_PARA_FINS_DE_PENSÃO_POR_MORTE.csv'

#### RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA

In [289]:
RELACAO = 'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [309]:
i = 0
SECRETARIA = ''
str_list_items = 'RF;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric():
        #print(s_item[0])
        #print(s_item[1:-3])
        #print(s_item[-3])
        #print(s_item[-2])
        #print(s_item[-1])
        #print(SECRETARIA)
        if s_item[-3].isnumeric():
            nome = str(' ').join(s_item[1:-3])
            str_list_items += f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}\n'
            #print(str_list_items)
        else:
            nome = str(' ').join(s_item[1:-2])
            str_list_items += f'{s_item[0]};{nome};;{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}\n'
            #print(str_list_items)
    i += 1

In [310]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-25\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_NOS_TERMOS_DA_LEI_8989_79.csv'